In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
plt.style.use('ggplot')
import re
import nltk
nltk.download('stopwords')
from nltk.util import ngrams
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from collections import  Counter
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM, 
                          Embedding, 
                          BatchNormalization,
                          Dense, 
                          TimeDistributed, 
                          Dropout, 
                          Bidirectional,
                          Flatten, 
                          GlobalMaxPool1D)
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
movies = pd.read_csv("/content/drive/My Drive/movie.csv")
genome_scores = pd.read_csv("/content/drive/My Drive/genome_scores.csv")
genome_tags = pd.read_csv("/content/drive/My Drive/genome_tags.csv")
plot = pd.read_csv("/content/drive/My Drive/wiki_movie_plots_deduped.csv")
reviews = pd.read_csv("/content/drive/My Drive/metacritic_reviews.csv")

In [13]:
plot.rename(columns={'Title': 'title'}, inplace=True)

In [14]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [15]:
movies['title'] = movies['title'].str.replace(r"\(.*\)","")

In [16]:
movies['title'] = movies['title'].str.rstrip()

In [17]:
df = pd.merge(movies, plot ,  how='left', left_on='title', right_on='title')

In [18]:
df = df[df['Plot'].notna()]

In [19]:
df = df.reset_index()

In [20]:
df

,index,movieId,title,genres,Release Year,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0,American,John Lasseter,"Tim Allen, Tom Hanks (voices)",animated film,https://en.wikipedia.org/wiki/Toy_Story,In a world where toys are living things who pr...
1,1,2,Jumanji,Adventure|Children|Fantasy,1995.0,American,Joe Johnston,"Robin Williams, Bonnie Hunt, Kirsten Dunst, Br...","family, fantasy",https://en.wikipedia.org/wiki/Jumanji_(film),"In 1869, near Brantford, New Hampshire, two br..."
2,2,3,Grumpier Old Men,Comedy|Romance,1995.0,American,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret, Soph...",comedy,https://en.wikipedia.org/wiki/Grumpier_Old_Men,The feud between Max (Walter Matthau) and John...
3,3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0,American,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...",drama,https://en.wikipedia.org/wiki/Waiting_to_Exhale,"""Friends are the People who let you be yoursel..."
4,4,5,Father of the Bride Part II,Comedy,1995.0,American,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...",comedy,https://en.wikipedia.org/wiki/Father_of_the_Br...,The film begins five years after the events of...
...,...,...,...,...,...,...,...,...,...,...,...
11988,29222,131152,The Fat Spy,Comedy,1966.0,American,Joseph Cates,"Phyllis Diller, Jack E. Leonard, Brian Donlevy...",comedy,https://en.wikipedia.org/wiki/The_Fat_Spy,"A mostly-deserted island, which is believed to..."
11989,29230,131168,Phoenix,Drama,1998.0,American,Danny Cannon,"Ray Liotta, Anjelica Huston, Anthony LaPaglia,...",crime,https://en.wikipedia.org/wiki/Phoenix_(1998_film),"In Phoenix, Arizona, Harry Collins is a cop wh..."
11990,29235,131180,Dead Rising: Watchtower,Action|Horror|Thriller,2015.0,Canadian,Zach Lipovsky,"Jesse Metcalfe, Dennis Haysbert, Virginia Madsen",zombie horror,https://en.wikipedia.org/wiki/Dead_Rising:_Wat...,Set between the events of Dead Rising 2 and De...
11991,29237,131237,What Men Talk About,Comedy,2010.0,Russian,Dmitry Dyashchenko,NaN,comedy,https://en.wikipedia.org/wiki/What_Men_Talk_About,"What Men Talk About? Of course, women. But als..."


In [21]:
reviews

,critic_name,media,title,review_date,individual_meta_score,text
0,M. E. Russell,Portland Oregonian,Hustle & Flow,NaN,100,"The writing, acting and filmmaking make Hustle..."
1,Owen Gleiberman,Entertainment Weekly,Hustle & Flow,NaN,91,The home-studio recording sequences in Hustle ...
2,Ann Hornaday,Washington Post,Hustle & Flow,NaN,90,"The performances are accomplished, but the rea..."
3,Stephanie Zacharek,Salon,Hustle & Flow,NaN,90,In a world of movies that try far too hard to ...
4,Jami Bernard,New York Daily News,Hustle & Flow,NaN,88,The feel-good movie of the summer. And the son...
...,...,...,...,...,...,...
246950,John Patterson,L.A. Weekly,The Quiet American,NaN,70,"Noyce has made a good-looking, intelligent sta..."
246951,Rick Groen,The Globe and Mail (Toronto),The Quiet American,NaN,63,The result is a rarity on the modern screen --...
246952,Jonathan Foreman,New York Post,The Quiet American,NaN,63,"It's a shame that, on top of everything else, ..."
246953,Jami Bernard,New York Daily News,The Quiet American,NaN,50,The movie adds nothing to the political dialog...


In [22]:
reviews = reviews.astype(str) 

In [23]:
reviews = reviews.groupby("title")['text'].apply(' '.join).reset_index()

In [24]:
reviews

,title,text
0,!Women Art Revolution,"These interviews form the backbone of !W.A.R.,..."
1,#Horror,Not every gamble works: The girls' intrusive B...
2,$9.99,"Using the droll, wise stories of Etgar Keret a..."
3,$pent,Has a warm and intimate feel that helps push i...
4,'71,"Swift and exciting, with no taste for the usua..."
...,...,...
12024,xXx: State of the Union,This is a B movie rooted in gut-level stirring...
12025,À Tout de Suite (Right Now),It's hard to imagine many films surpassing or ...
12026,À cause d'un garçon,"While Cazeneuve's story is about gay love, it ..."
12027,Æon Flux,"It's all so geekily gorgeous, it hardly matter..."


In [25]:
df = pd.merge(df, reviews ,  how='left', left_on='title', right_on='title')

In [26]:
df = df[df['text'].notna()]

In [27]:
df

,index,movieId,title,genres,Release Year,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,text
0,0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0,American,John Lasseter,"Tim Allen, Tom Hanks (voices)",animated film,https://en.wikipedia.org/wiki/Toy_Story,In a world where toys are living things who pr...,"With ""instant classic"" written all over it, To..."
1,1,2,Jumanji,Adventure|Children|Fantasy,1995.0,American,Joe Johnston,"Robin Williams, Bonnie Hunt, Kirsten Dunst, Br...","family, fantasy",https://en.wikipedia.org/wiki/Jumanji_(film),"In 1869, near Brantford, New Hampshire, two br...",The result is a thrill ride with enough plunge...
2,2,3,Grumpier Old Men,Comedy|Romance,1995.0,American,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret, Soph...",comedy,https://en.wikipedia.org/wiki/Grumpier_Old_Men,The feud between Max (Walter Matthau) and John...,Grumpier is a welcome continuation that leaves...
4,4,5,Father of the Bride Part II,Comedy,1995.0,American,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...",comedy,https://en.wikipedia.org/wiki/Father_of_the_Br...,The film begins five years after the events of...,nan What Meyers and Shyer have accomplished is...
5,5,6,Heat,Action|Crime|Thriller,1972.0,American,Paul Morrissey,"Joe Dallesandro, Sylvia Miles",drama,https://en.wikipedia.org/wiki/Heat_(1972_film),Joey Davis is an unemployed former child star ...,Stunningly made and incisively acted by a larg...
...,...,...,...,...,...,...,...,...,...,...,...,...
11970,29116,130622,The Circle,Documentary|Drama,1925.0,American,Frank Borzage,"Eleanor Boardman, Malcolm McGregor",romance drama,https://en.wikipedia.org/wiki/The_Circle_(1925...,"In the 1890s, young Lady Catherine (Joan Crawf...",The Circle is very much a plea for the preserv...
11971,29117,130622,The Circle,Documentary|Drama,2017.0,American,James Ponsoldt,James Ponsoldt (director/screenplay); Tom Hank...,"sci-fi, drama, thriller",https://en.wikipedia.org/wiki/The_Circle_(2017...,"When her car breaks down, Mae Holland contacts...",The Circle is very much a plea for the preserv...
11978,29162,131013,Get Hard,Comedy,2015.0,American,Etan Cohen,Will Ferrell\r\nKevin Hart\r\nAlison Brie\r\nE...,comedy,https://en.wikipedia.org/wiki/Get_Hard,James King is an extremely wealthy hedge fund ...,It matches up two comic actors and instead of ...
11987,29207,131122,Love Exposure,Action|Comedy|Drama|Romance,2008.0,Japanese,Shion Sono,"Atsuro Watabe, Itsuji Itao, Mami Nakamura, Hik...","romance, comedy, drama, art house & internatio...",https://en.wikipedia.org/wiki/Love_Exposure,The story follows Yū Honda (Takahiro Nishijima...,"The movie’s invigorating discourse on sin, lus..."


In [28]:
 df.drop(df.columns.difference(['Plot','title', 'text', 'movieId']), 1, inplace=True)

In [29]:
df = df.reset_index()

In [30]:
df = df.drop(['index'], axis = 1)

In [31]:
df.head()

,movieId,title,Plot,text
0,1,Toy Story,In a world where toys are living things who pr...,"With ""instant classic"" written all over it, To..."
1,2,Jumanji,"In 1869, near Brantford, New Hampshire, two br...",The result is a thrill ride with enough plunge...
2,3,Grumpier Old Men,The feud between Max (Walter Matthau) and John...,Grumpier is a welcome continuation that leaves...
3,5,Father of the Bride Part II,The film begins five years after the events of...,nan What Meyers and Shyer have accomplished is...
4,6,Heat,Joey Davis is an unemployed former child star ...,Stunningly made and incisively acted by a larg...


In [32]:
genome = pd.merge(genome_scores, genome_tags ,  how='left', left_on='tagId', right_on='tagId')

In [33]:
genome

,movieId,tagId,relevance,tag
0,1,1,0.02500,007
1,1,2,0.02500,007 (series)
2,1,3,0.05775,18th century
3,1,4,0.09675,1920s
4,1,5,0.14675,1930s
...,...,...,...,...
11709763,131170,1124,0.58775,writing
11709764,131170,1125,0.01075,wuxia
11709765,131170,1126,0.01575,wwii
11709766,131170,1127,0.11450,zombie


In [34]:
genome = genome.groupby("movieId").agg(lambda x: x.tolist())

In [35]:
genome = genome.drop(['tagId'], axis = 1)

In [36]:
genome = genome.reset_index()

In [37]:
genome

,movieId,relevance,tag
0,1,"[0.025, 0.025, 0.05775, 0.09675, 0.14675, 0.21...","[007, 007 (series), 18th century, 1920s, 1930s..."
1,2,"[0.03975, 0.04375, 0.03775, 0.048, 0.11025, 0....","[007, 007 (series), 18th century, 1920s, 1930s..."
2,3,"[0.0435, 0.05475, 0.027999999999999997, 0.077,...","[007, 007 (series), 18th century, 1920s, 1930s..."
3,4,"[0.03725, 0.0395, 0.03675, 0.031, 0.06825, 0.0...","[007, 007 (series), 18th century, 1920s, 1930s..."
4,5,"[0.042, 0.05275, 0.05925, 0.03675, 0.07525, 0....","[007, 007 (series), 18th century, 1920s, 1930s..."
...,...,...,...
10376,130578,"[0.154, 0.04025, 0.01975, 0.0185, 0.0285, 0.02...","[007, 007 (series), 18th century, 1920s, 1930s..."
10377,130840,"[0.024, 0.03025, 0.02425, 0.09375, 0.055999999...","[007, 007 (series), 18th century, 1920s, 1930s..."
10378,131013,"[0.042, 0.03175, 0.017, 0.055999999999999994, ...","[007, 007 (series), 18th century, 1920s, 1930s..."
10379,131168,"[0.05975, 0.10375, 0.1785, 0.14875, 0.2775, 0....","[007, 007 (series), 18th century, 1920s, 1930s..."


In [38]:
df = pd.merge(df, genome ,  how='left', left_on='movieId', right_on='movieId')

In [39]:
df

,movieId,title,Plot,text,relevance,tag
0,1,Toy Story,In a world where toys are living things who pr...,"With ""instant classic"" written all over it, To...","[0.025, 0.025, 0.05775, 0.09675, 0.14675, 0.21...","[007, 007 (series), 18th century, 1920s, 1930s..."
1,2,Jumanji,"In 1869, near Brantford, New Hampshire, two br...",The result is a thrill ride with enough plunge...,"[0.03975, 0.04375, 0.03775, 0.048, 0.11025, 0....","[007, 007 (series), 18th century, 1920s, 1930s..."
2,3,Grumpier Old Men,The feud between Max (Walter Matthau) and John...,Grumpier is a welcome continuation that leaves...,"[0.0435, 0.05475, 0.027999999999999997, 0.077,...","[007, 007 (series), 18th century, 1920s, 1930s..."
3,5,Father of the Bride Part II,The film begins five years after the events of...,nan What Meyers and Shyer have accomplished is...,"[0.042, 0.05275, 0.05925, 0.03675, 0.07525, 0....","[007, 007 (series), 18th century, 1920s, 1930s..."
4,6,Heat,Joey Davis is an unemployed former child star ...,Stunningly made and incisively acted by a larg...,"[0.02825, 0.0255, 0.0185, 0.0455, 0.09575, 0.0...","[007, 007 (series), 18th century, 1920s, 1930s..."
...,...,...,...,...,...,...
5957,130622,The Circle,"In the 1890s, young Lady Catherine (Joan Crawf...",The Circle is very much a plea for the preserv...,NaN,NaN
5958,130622,The Circle,"When her car breaks down, Mae Holland contacts...",The Circle is very much a plea for the preserv...,NaN,NaN
5959,131013,Get Hard,James King is an extremely wealthy hedge fund ...,It matches up two comic actors and instead of ...,"[0.042, 0.03175, 0.017, 0.055999999999999994, ...","[007, 007 (series), 18th century, 1920s, 1930s..."
5960,131122,Love Exposure,The story follows Yū Honda (Takahiro Nishijima...,"The movie’s invigorating discourse on sin, lus...",NaN,NaN


In [40]:
df = df[df['relevance'].notna()]

In [41]:
df = df[df['tag'].notna()]

In [42]:
df = df[df['Plot'].notna()]

In [43]:
df = df.reset_index()

In [44]:
len(df['relevance'])

4561

In [45]:
df

,index,movieId,title,Plot,text,relevance,tag
0,0,1,Toy Story,In a world where toys are living things who pr...,"With ""instant classic"" written all over it, To...","[0.025, 0.025, 0.05775, 0.09675, 0.14675, 0.21...","[007, 007 (series), 18th century, 1920s, 1930s..."
1,1,2,Jumanji,"In 1869, near Brantford, New Hampshire, two br...",The result is a thrill ride with enough plunge...,"[0.03975, 0.04375, 0.03775, 0.048, 0.11025, 0....","[007, 007 (series), 18th century, 1920s, 1930s..."
2,2,3,Grumpier Old Men,The feud between Max (Walter Matthau) and John...,Grumpier is a welcome continuation that leaves...,"[0.0435, 0.05475, 0.027999999999999997, 0.077,...","[007, 007 (series), 18th century, 1920s, 1930s..."
3,3,5,Father of the Bride Part II,The film begins five years after the events of...,nan What Meyers and Shyer have accomplished is...,"[0.042, 0.05275, 0.05925, 0.03675, 0.07525, 0....","[007, 007 (series), 18th century, 1920s, 1930s..."
4,4,6,Heat,Joey Davis is an unemployed former child star ...,Stunningly made and incisively acted by a larg...,"[0.02825, 0.0255, 0.0185, 0.0455, 0.09575, 0.0...","[007, 007 (series), 18th century, 1920s, 1930s..."
...,...,...,...,...,...,...,...
4556,5954,130520,Home,The Shamanov family lives in a two-story house...,"The combination of Home’s layered message, fun...","[0.05875, 0.02325, 0.0215, 0.04425, 0.05, 0.02...","[007, 007 (series), 18th century, 1920s, 1930s..."
4557,5955,130578,The Gunman,Jim Terrier (Sean Penn) is a former special fo...,"Basically, The Gunman is a movie that asks aud...","[0.154, 0.04025, 0.01975, 0.0185, 0.0285, 0.02...","[007, 007 (series), 18th century, 1920s, 1930s..."
4558,5956,130578,The Gunman,Jim Terrier (Sean Penn) is a former special fo...,"Basically, The Gunman is a movie that asks aud...","[0.154, 0.04025, 0.01975, 0.0185, 0.0285, 0.02...","[007, 007 (series), 18th century, 1920s, 1930s..."
4559,5959,131013,Get Hard,James King is an extremely wealthy hedge fund ...,It matches up two comic actors and instead of ...,"[0.042, 0.03175, 0.017, 0.055999999999999994, ...","[007, 007 (series), 18th century, 1920s, 1930s..."


In [10]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import TimeDistributed, Dense, LSTM, Embedding, Dropout, Bidirectional, GlobalMaxPool1D
from tensorflow.python.keras.models import Model, Sequential
from keras.utils.np_utils import to_categorical# -------- other packages ------------------ #
import sys
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import pickle

np.set_printoptions(threshold=sys.maxsize)
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

df = pd.read_csv("/content/drive/My Drive/wikidata.csv", delimiter = "\t", names = ["Sentence", "Keyword"])
df['Sentence'] = df['Sentence'].astype(str)
df['Keyword'] = df['Keyword'].astype(str)

def hasNumbers(inputString):
	return any(char.isdigit() for char in inputString)

def tag_keywords(all_keywords):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts([all_keywords])
	all_keywords = [i for i in tokenizer.word_index.keys()]
	all_keywords = list(set(all_keywords))
	return all_keywords

# sentence cleaning
df['Sentence'] = df['Sentence'].apply(lambda x: x.replace(" – TechCrunch",""))
df['Keyword'] = df['Keyword'].apply(lambda x: tag_keywords(x))

sentence_column = []
keyword_column = []
for index, row in df.iterrows():
	new_keywords = []
	sentence = row['Sentence']
	keywords = row['Keyword']
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts([sentence])
	tokens = [i for i in tokenizer.word_index.keys()]
	for i in tokens:
		if i in keywords:
			if not hasNumbers(i):
				new_keywords.append(1)
		else:
			new_keywords.append(0)
	if sum(new_keywords) != 0:
		sentence_column.append(sentence)
		keyword_column.append(new_keywords)


tokenizer = Tokenizer(oov_token = "<UNK>")
tokenizer.fit_on_texts(sentence_column)
X = tokenizer.texts_to_sequences(sentence_column)
X = pad_sequences(X, padding = "post", truncating = "post", maxlen = 500, value = 0)
y = pad_sequences(keyword_column, padding = "post", truncating = "post", maxlen = 500, value = 0)
y = [to_categorical(i, num_classes = 2) for i in y]
embeddings_index = {}
# e_dat is an embedding file(glove)
f = open('/content/drive/My Drive/glove.6B.200d.txt','r')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype = "float32")
	embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

EMBEDDING_DIM = 200
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
# Model creation
model = Sequential()
model.add(Embedding(len(word_index) + 1, 200, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences = True, recurrent_dropout = 0.3)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128, return_sequences = True, recurrent_dropout = 0.1)))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(2, activation = "softmax")))
model.compile(loss="categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.fit(X_train, np.array(y_train), batch_size = 32, epochs = 5, validation_split = 0.1)

# model save
model_json = model.to_json()
with open("/content/drive/My Drive/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("/content/drive/My Drive/model.h5")
pickle.dump(tokenizer, open("tokenizer.pickle","wb"))
print("Saved model to disk")

json_file = open('/content/drive/My Drive/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/model.h5")
print("Loaded model from disk")

test_output = loaded_model.predict(X_test)
test_output = np.argmax(test_output, axis = -1)


flattened_actual = (np.argmax(np.array(y_test), axis = -1)).flatten()
flattened_output = test_output.flatten()

print(classification_report(flattened_actual, flattened_output))




In [ ]:
try:
  while True:
    input_ = input("Type in a headline:")
    new_t = Tokenizer()
    new_t.fit_on_texts([input_])
    tokens = [i for i in new_t.word_index.keys()]
#		print(tokens)
    actual_tokens = new_t.texts_to_sequences([input_])
    inv_map_tokens = {v: k for k, v in new_t.word_index.items()}
    actual_tokens = [inv_map_tokens[i] for i in actual_tokens[0]]
    tokens = actual_tokens
    input_ = tokenizer.texts_to_sequences([input_])
    input_ = pad_sequences(input_, padding = "post", truncating = "post", maxlen = 500, value = 0)
    output = model.predict([input_])
    output = np.argmax(output, axis = -1)
    where_ = np.where(output[0] == 1)[0]
    output_keywords = np.take(tokens, where_)
    lem= WordNetLemmatizer()
    output_keywords = [lem.lemmatize(word) for word in output_keywords if not word in set(stopwords.words('english'))]
    output_keywords = list(set(output_keywords))
#		print(tokens)
#		print(output)
    print(output_keywords)
    print(len(output_keywords))
except KeyboardInterrupt:
	pass

In [38]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    lem= WordNetLemmatizer()
    text = text.split()
    text = [lem.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

In [40]:
df['text'] = df['text'].apply(lambda x: clean_text(x))
df['Plot'] = df['Plot'].apply(lambda x: clean_text(x))

In [41]:
df['plot'] = df['Plot'] + df['text']

In [42]:
df.head()

,index,movieId,title,Plot,text,relevance,tag,plot
0,0,1,Toy Story,world toy living thing pretend lifeless human ...,instant classic written toy story first fullle...,"[0.025, 0.025, 0.05775, 0.09675, 0.14675, 0.21...","[007, 007 (series), 18th century, 1920s, 1930s...",world toy living thing pretend lifeless human ...
1,1,2,Jumanji,near brantford new hampshire two brother bury ...,result thrill ride enough plunge turn loopthel...,"[0.03975, 0.04375, 0.03775, 0.048, 0.11025, 0....","[007, 007 (series), 18th century, 1920s, 1930s...",near brantford new hampshire two brother bury ...
2,2,3,Grumpier Old Men,feud max walter matthau john jack lemmon coole...,grumpier welcome continuation leaf wanting ano...,"[0.0435, 0.05475, 0.027999999999999997, 0.077,...","[007, 007 (series), 18th century, 1920s, 1930s...",feud max walter matthau john jack lemmon coole...
3,3,5,Father of the Bride Part II,film begin five year event first one george ba...,nan meyers shyer accomplished create pleasant ...,"[0.042, 0.05275, 0.05925, 0.03675, 0.07525, 0....","[007, 007 (series), 18th century, 1920s, 1930s...",film begin five year event first one george ba...
4,4,6,Heat,joey davis unemployed former child star suppor...,stunningly made incisively acted large terrifi...,"[0.02825, 0.0255, 0.0185, 0.0455, 0.09575, 0.0...","[007, 007 (series), 18th century, 1920s, 1930s...",joey davis unemployed former child star suppor...


In [43]:
plot = df['plot'].values
score = df['relevance'].values

In [44]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(plot)
vocab_length = len(word_tokenizer.word_index) + 1

In [45]:
longest_train = max(plot, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(longest_train))
padded_sentences = pad_sequences(word_tokenizer.texts_to_sequences(plot), 500, padding='post')

In [46]:
embeddings_index = {}
# e_dat is an embedding file(glove)
f = open('/content/drive/My Drive/glove.6B.200d.txt','r')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype = "float32")
	embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [47]:
EMBEDDING_DIM = 200
word_index = word_tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [48]:
X_train, X_test, y_train, y_test = train_test_split( padded_sentences, score, test_size=0.1, random_state=0)

In [58]:
def BLSTM():
    model = Sequential()
    model.add(Embedding(input_dim=len(word_index)+1, 
                        output_dim=200, 
                        weights = [embedding_matrix],
                        input_length = 500))
    model.add(Bidirectional(LSTM(500, return_sequences = True, recurrent_dropout=0.2)))
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2048, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1128, activation = 'sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [50]:
def build_lrfn(lr_start=0.00001, lr_max=0.000075, 
               lr_min=0.000001, lr_rampup_epochs=20, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    
    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

lrfn = build_lrfn()
lr_schedule = LearningRateScheduler(lrfn, verbose=1)

In [59]:
model = BLSTM()
checkpoint = ModelCheckpoint(
    '/content/drive/My Drive/model_tag.h5', 
    monitor = 'val_loss', 
    verbose = 1, 
    save_best_only = True
)

history = model.fit(
    X_train, 
    np.stack(y_train),
    epochs = 7,
    batch_size = 64,
    validation_data = [X_test, y_test],
    verbose = 1,
    callbacks = [checkpoint, lr_schedule]
)


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/7
65/65 [==============================] - ETA: 0s - loss: 0.0481 - accuracy: 0.0027

ValueError: ignored

In [ ]:
y_train